In [137]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [124]:
df_jan = pd.read_parquet('../data/fhv_tripdata_2021-01.parquet')
df_feb = pd.read_parquet('../data/fhv_tripdata_2021-02.parquet')

In [125]:
df.head()

,PUlocationID,DOlocationID
0,-1.0,-1.0
1,-1.0,-1.0
2,-1.0,72.0
3,-1.0,61.0
4,-1.0,71.0


### Question 1

In [126]:
df_jan.shape[0]

1154112

### Question 2

In [127]:
(df_jan['dropOff_datetime'] - df_jan['pickup_datetime']).mean()

Timedelta('0 days 00:19:10.033445627')

### Filter rows

In [128]:
df_jan['trip_time_minutes'] = (df_jan['dropOff_datetime'] - df_jan['pickup_datetime']).dt.seconds / 60
df_jan = df_jan[df_jan['trip_time_minutes'].between(1, 60)]

df_feb['trip_time_minutes'] = (df_feb['dropOff_datetime'] - df_feb['pickup_datetime']).dt.seconds / 60
df_feb = df_feb[df_feb['trip_time_minutes'].between(1, 60)]

### Question 3

In [129]:
df_jan[['PUlocationID', 'DOlocationID']] = df_jan[['PUlocationID', 'DOlocationID']].fillna(-1)
df_feb[['PUlocationID', 'DOlocationID']] = df_feb[['PUlocationID', 'DOlocationID']].fillna(-1)

In [130]:
df_jan['PUlocationID'].value_counts(normalize=True)*100

-1.0      83.527548
 221.0     0.750550
 206.0     0.612423
 129.0     0.484659
 115.0     0.367796
            ...    
 111.0     0.000451
 27.0      0.000360
 34.0      0.000270
 2.0       0.000180
 110.0     0.000090
Name: PUlocationID, Length: 262, dtype: float64

### Create matrixes

In [103]:
train_len = df_jan.shape[0]

In [132]:
df = pd.concat((df_jan, df_feb), ignore_index=True)

In [112]:
df['PUlocationID'] = df['PUlocationID'].astype(str)
df['DOlocationID'] = df['DOlocationID'].astype(str)

In [113]:
D = df[['PUlocationID', 'DOlocationID']].to_dict(orient='records')

In [118]:
v = DictVectorizer(sparse=True)

In [119]:
X = v.fit_transform(D)

### Question 4

In [120]:
X.shape[1]

526

### Question 5

In [145]:
model = LinearRegression()
model = model.fit(X[:train_len], df['trip_time_minutes'].iloc[:train_len])

In [146]:
pred = model.predict(X[:train_len])
np.sqrt(mean_squared_error(df['trip_time_minutes'].iloc[:train_len], pred))

10.528643590004952

### Question 6

In [147]:
pred = model.predict(X[train_len:])

In [148]:
np.sqrt(mean_squared_error(df['trip_time_minutes'].iloc[train_len:], pred))

11.014590537467848